In [1]:
import os
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Embedding, TextVectorization,LSTM,Dropout,Bidirectional
from keras.models import Sequential
from keras.callbacks import TensorBoard

C:\Users\HP\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('train.csv')
#SPLIT OUT OUR DATA INTO COMMENTS AND FEATURES
x=df['comment_text']
y=df[df.columns[2:]].values #each comment has a vector now(array)
#under x we have all our coments
#under y we have all our labels


In [3]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
## VECTORIZATION IS TOKENIZATION

In [4]:
max_features=200000 #number of words in the vocab dictionary
vectorizer=TextVectorization(max_tokens=max_features,
                             output_sequence_length=1800,#maximum lenghth in token
                             output_mode='int')#map every single word to integer value

vectorizer.adapt(x.values)#teach vocab
vectorized_text = vectorizer(x.values)#change the sentence to a seq of numbers.

In [5]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()+

In [6]:
train=dataset.take(int(len(dataset)*.7))#take 70% of dataset
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))
#skip function skips partictular patrtition of dataset


In [7]:
model=Sequential() 
# Create the embedding Layer
model.add(Embedding(max_features+1,32))
# Implement a Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32,activation='tanh')))
# Feature extractor Fully connected Layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
# Final Layer
model.add(Dense(6,activation='sigmoid'))

In [8]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam',metrics='accuracy')
print(model.summary())

#Training Model
model.fit(train,epochs=1,validation_data=val, callbacks=tb_callback)
#epochs = no of times the model is trained
model.save("modelV2.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        